<a href="https://colab.research.google.com/github/willymenacho/HTML/blob/main/MESH_GENERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ERROR: File or directory already exists: '/usr/local/miniconda'
If you want to update an existing installation, use the -u option.


In [5]:
# Descargar e instalar Miniconda (conda base)
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local/miniconda

import sys
import os
sys.path.append('/usr/local/miniconda/lib/python3.10/site-packages')
os.environ['PATH'] = '/usr/local/miniconda/bin:' + os.environ['PATH']

# Aceptar Terms of Service para evitar preguntas interactivas
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Instalar NanoShaper en el entorno base de conda
!conda install electrostatics::nanoshaper

# Verificar instalación
NANOSHAPER_PATH = "/usr/local/miniconda/bin/NanoShaper"


ERROR: File or directory already exists: '/usr/local/miniconda'
If you want to update an existing installation, use the -u option.
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
2 channel Terms of Service accepted
Channels:
 - defaults
 - electrostatics
Platform: linux-64
Solving environment: \ | done

# All requested packages already installed.



In [3]:
# --- Montar Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!NanoShaper

/bin/bash: line 1: NanoShaper: command not found


In [15]:
import numpy as np
import os
import shutil

dire_base = '/content/drive/MyDrive/SIMULACION_PYGBE/test/casos1000'

carpetas_principales=['0_d_original','1_d_aumentada','3_open_RBD']

carpeta_casos_0=['head_omicron1','RBM_omicron0'] #['head_omicron0','head_omicron1','head_WT0','head_WT1','RBM_omicron0','RBM_omicron1','RBM_WT0','RBM_WT1','head_omicron1_ant','head_omicron1_pos']
carpeta_casos_1=['head_WT0','RBM_WT1'] #['head_omicron0','head_omicron1','head_WT0','head_WT1','RBM_omicron0','RBM_omicron1','RBM_WT0','RBM_WT1','head_omicron0_substitute']
carpeta_casos_3=['omicron1_116','wt0_178']
                #['omicron0_191','omicron0_60','omicron1_165','omicron1_78','wt0_186','wt1_190','omicron0_207','omicron0_89','omicron1_27',
                # 'wt0_138','wt0_226','wt1_82','omicron0_222','omicron1_116','omicron1_299','wt0_175','wt0_278','omicron0_32','omicron1_139',
                # 'omicron1_68','wt0_178','wt1_125']

id_key=['move100A','move1000A']


prm_template = """Operative_Mode = normal
Compute_Vertex_Normals = true
Save_Mesh_MSMS_Format = true
Pocket_Radius_Big = 3.0
Pocket_Radius_Small = 1.4
Num_Wat_Pocket = 2
Tri2Balls = false
Grid_scale = 1.5
Grid_perfil = 90.0
XYZR_FileName = {xyzr}
Build_epsilon_maps  = false
Build_status_map = true
Surface = ses
Smooth_Mesh = true
Number_thread = 32
Skin_Surface_Parameter = 0.45
Blobbyness = -2.5
Surface_File_Name = triangulatedSurf.off
Save_Mesh_MSMS_Format = true
Cavity_Detection_Filling = true
Conditional_Volume_Filling_Value = 11.4
Keep_Water_Shaped_Cavities = false
Probe_Radius = 1.4
Accurate_Triangulation = true
Triangulation = true
Check_duplicated_vertices = true
Save_Status_map = false
Save_PovRay = false
Max_mesh_auxiliary_grid_size = 100
Max_mesh_patches_per_auxiliary_grid_cell = 250
Max_mesh_auxiliary_grid_2d_size = 100
Max_mesh_patches_per_auxiliary_grid_2d_cell = 250
Max_ses_patches_auxiliary_grid_size = 100geometry
Max_ses_patches_per_auxiliary_grid_cell = 400
Max_ses_patches_auxiliary_grid_2d_size = 40
Max_ses_patches_per_auxiliary_grid_2d_cell = 400
Max_skin_patches_auxiliary_grid_size = 100
Max_skin_patches_per_auxiliary_grid_cell = 400
Max_skin_patches_auxiliary_grid_2d_size = 50
Max_skin_patches_per_auxiliary_grid_2d_cell = 400
"""



# Función para crear xyzr desde pqr
def create_xyzr_from_pqr(pqr_path, xyzr_path):
    with open(pqr_path, 'r') as infile, open(xyzr_path, 'w') as outfile:
        for line in infile:
            if line.startswith('ATOM') or line.startswith('HETATM'):
                parts = line.split()
                try:
                    x, y, z, radius = parts[5], parts[6], parts[7], parts[9]
                    outfile.write(f"{x} {y} {z} {radius}\n")
                except IndexError:
                    print(f"Error parsing line in {pqr_path}: {line.strip()}")

for raiz in carpetas_principales:
    #comentar esta seccion si no es el caso completo
    if raiz == '0_d_original':
        carpeta_casos=carpeta_casos_0
    elif raiz == '1_d_aumentada':
        carpeta_casos=carpeta_casos_1
    elif raiz == '3_open_RBD':
        carpeta_casos=carpeta_casos_3
    #comentar esta seccion si no es el caso competo


    for rama in carpeta_casos:
      for key in id_key:


        for mol in ['prot', 'gly']:
          geometry_path = f"{dire_base}/{raiz}/{rama}/{key}/geometry"
          path_caso=f"{dire_base}/{raiz}/{rama}/{key}"
          pqr_path = f"{dire_base}/{raiz}/{rama}/{key}/{mol}.pqr"
          xyzr_path = f"{dire_base}/{raiz}/{rama}/{key}/{mol}.xyzr"
          prm_path = f"{dire_base}/{raiz}/{rama}/{key}/{mol}.prm"
          create_xyzr_from_pqr(pqr_path, xyzr_path)
          prm_content = prm_template.format(xyzr=os.path.basename(xyzr_path))
          with open(prm_path, 'w') as f:
              f.write(prm_content)

          current_dir = os.getcwd()
          os.chdir(os.path.dirname(prm_path))
          print(os.getcwd(),current_dir)
          print({os.path.basename(prm_path)})

          print(f"[INFO] Ejecutando NanoShaper en {prm_path}")
          os.system(f"{NANOSHAPER_PATH} {os.path.basename(prm_path)}")


          # 4. Mover .vert y .face a geometry y renombrar
          for ext in ['vert', 'face']:
              src = f"{path_caso}/triangulatedSurf.{ext}"
              dst = os.path.join(geometry_path, f"{mol}.{ext}")
              if os.path.exists(src):
                  shutil.move(src, dst)
                  print(f"[OK] Movido: {src} → {dst}")
              else:
                  print(f"[WARN] No se encontró {src}")

          # 5. Volver al directorio original
          os.chdir(current_dir)



/content/drive/MyDrive/SIMULACION_PYGBE/test/casos8/0_d_original/head_omicron1/move2A /content/drive/MyDrive/SIMULACION_PYGBE/test/casos1000/0_d_original/head_omicron1/move100A
{'prot.prm'}
[INFO] Ejecutando NanoShaper en /content/drive/MyDrive/SIMULACION_PYGBE/test/casos8/0_d_original/head_omicron1/move2A/prot.prm
[OK] Movido: /content/drive/MyDrive/SIMULACION_PYGBE/test/casos8/0_d_original/head_omicron1/move2A/triangulatedSurf.vert → /content/drive/MyDrive/SIMULACION_PYGBE/test/casos8/0_d_original/head_omicron1/move2A/geometry/prot.vert
[OK] Movido: /content/drive/MyDrive/SIMULACION_PYGBE/test/casos8/0_d_original/head_omicron1/move2A/triangulatedSurf.face → /content/drive/MyDrive/SIMULACION_PYGBE/test/casos8/0_d_original/head_omicron1/move2A/geometry/prot.face
/content/drive/MyDrive/SIMULACION_PYGBE/test/casos8/0_d_original/head_omicron1/move2A /content/drive/MyDrive/SIMULACION_PYGBE/test/casos1000/0_d_original/head_omicron1/move100A
{'gly.prm'}
[INFO] Ejecutando NanoShaper en /conte